In [ ]:
cd C:\Users\KESJ\Documents\projects\Production Migration Project\csv

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [ ]:
years=['0405','0506','0607','0708','0809','0910','1011']
years[4:]

In [ ]:
years=['0405','0506','0607','0708','0809','0910','1011']


for y in years[6:]:
    print "Starting with ",y
    inflow = pd.read_csv("countyinflow"+y+".csv",
                         dtype={'State_Code_Dest':np.str,
                                'County_Code_Dest':np.str,
                                'State_Code_Origin':np.str,
                                'County_Code_Origin':np.str,
                                'State_Abbrv':np.str,
                                'County_Name':np.str})

    outflow = pd.read_csv("countyoutflow"+y+".csv",
                         dtype={'State_Code_Dest':np.str,
                                'County_Code_Dest':np.str,
                                'State_Code_Origin':np.str,
                                'County_Code_Origin':np.str,
                                'State_Abbrv':np.str,
                                'County_Name':np.str})


    fips = get_all_fips(inflow)

    df = net_df()

    for i,f in enumerate(fips):
        print y,'-',i,'/',len(fips),':',f
        df = df.append(get_net_change_for_fips(inflow,outflow,f[0],f[1]), ignore_index=True)
        sys.stdout.flush()

    df.sort([df.columns[0],df.columns[1]],inplace=True,ascending=True)
    df.to_csv("countynetflow2"+y+".csv")

# now dealing with 0910

In [ ]:
years=['0405','0506','0607','0708','0809','0910','1011']


y = years[5]
print "Starting with ",y
inflow = pd.read_csv("countyinflow"+y+".csv",
                         dtype={'State_Code_Dest':np.str,
                                'County_Code_Dest':np.str,
                                'State_Code_Origin':np.str,
                                'County_Code_Origin':np.str,
                                'State_Abbrv':np.str,
                                'County_Name':np.str})
inflow[['State_Code_Dest','State_Code_Origin']]=inflow[['State_Code_Dest','State_Code_Origin']].applymap(lambda x: prependZeros(x,2))
inflow[['County_Code_Origin','County_Code_Dest']]=inflow[['County_Code_Origin','County_Code_Dest']].applymap(lambda x: prependZeros(x))

outflow = pd.read_csv("countyoutflow"+y+".csv",
                         dtype={'State_Code_Dest':np.str,
                                'County_Code_Dest':np.str,
                                'State_Code_Origin':np.str,
                                'County_Code_Origin':np.str,
                                'State_Abbrv':np.str,
                                'County_Name':np.str})

outflow[['County_Code_Origin','County_Code_Dest']]=outflow[['County_Code_Origin','County_Code_Dest']].applymap(lambda x: prependZeros(x))

fips = get_all_fips(inflow)

df = net_df()

for i,f in enumerate(fips):
    print y,'-',i,'/',len(fips),':',f
    df = df.append(get_net_change_for_fips(inflow,outflow,f[0],f[1]), ignore_index=True)
    sys.stdout.flush()

df.sort([df.columns[0],df.columns[1]],inplace=True,ascending=True)
df.to_csv("countynetflow2"+y+".csv")

# inflow needs the county_code_origin prepended by '00' or '0'

In [ ]:
def prependZeros(inValue,desLen=3):
    while (len(inValue)<desLen):
        inValue='0'+inValue
    return inValue

In [ ]:
prependZeros('13',5)

In [ ]:
fips = get_all_fips(inflow)

df = net_df()

for i,f in enumerate(fips):
    print y,'-',i,'/',len(fips),':',f
    df = df.append(get_net_change_for_fips(inflow,outflow,f[0],f[1]), ignore_index=True)
    sys.stdout.flush()

df.sort([df.columns[0],df.columns[1]],inplace=True,ascending=True)
df.to_csv("countynetflow2"+y+".csv")

In [ ]:
[a for a in outflow['County_Code_Origin'].unique() if len(a)!=3]

In [ ]:
for i, f in enumerate(fips):
    print i, f

#AJ Rader --> putting all the PIF data (from pcadss together)
desire is to create a single dataframe that has initial PIF values for each year, final values for each year and change over that year.
the amounts reported in PIF20xxYY correspond to the total number in force at the end of month YY and year 20xx.

In [ ]:
pwd

In [ ]:
yearPeriods = ['0405','0506','0607','0708','0809','0910','1011','1112']

In [ ]:
for year in yearPeriods:
    print year
    pdf = pd.read_csv('pifchange'+year+'.csv',
                      dtype={'FIPS_CD':str, 'COUNTY':np.str,
                            'STATE':np.str, 'POSTL_ST_CD':np.str})
    pdf.drop('PIF_NET_CHANGE',1,inplace=True) #drop net change column
    #print np.shape(pdf), pdf.columns[:2]
    if int(year)== 405:
        pif_df2 = pdf.copy() #mv initial to 2nd version
    else:
        pif_df2 = pd.merge(pif_df2,pdf,on='FIPS_CD')
    
print "this dataframe has the following dimensions", np.shape(pif_df2)

In [ ]:
# identify repeat columns and remove them from the dataframe
dupColumn = []
#dupColumnY = []
#dColumnBase = []
nrows = len(pif_df2)
for c in pif_df2.columns:
    if c.endswith('_x'):
        dupColumn.append(c)
        # check if they match each other (sanity check)
        c2 = c[:-2]+"_y"
        #dupColumnY.append(c2)
        #dColumnBase.append(c[:-2])
        print "Do columns {0} and {1} match? {2}".format(c,c2,sum((pif_df2[c]-pif_df2[c2])==0)==nrows)

ydupCols = [c[:-2]+'_y' for c in dupColumn]
dupColDict = {}
for c in dupColumn:
    dupColDict[c]= c[:-2]
#dColumnBase = [c[:-2] for c in dupColumn]
pif_df2.drop(ydupCols,1,inplace=True)

In [ ]:
pif_df2.rename(columns=dupColDict,inplace=True)

In [ ]:
pif_df2.head()

In [ ]:
### SAVE full dataset
pif_df2.to_csv('pif_200410to201210.csv')

In [ ]:
pif_time = pif_df2.T

In [ ]:
#make a dictionary out of the fipsCodes for column headings
fipsCodes = pif_time.loc['FIPS_CD',:].values
fipsDict = {}
for i in xrange(0,len(fipsCodes)):
    fipsDict[i]=fipsCodes[i]

pif_time.drop('FIPS_CD',0,inplace=True)
pif_time.rename(columns=fipsDict,inplace=True)
#.values


In [ ]:
pd.datetime?

In [ ]:
## convert PIF20xxYY to date
date = pd.to_datetime([a[3:] for a in pif_time.index],format='%Y%m')
date

In [ ]:
pif_time.index = date

In [ ]:
%pylab inline

In [ ]:
yearEnd = pif_time.resample('AS-JAN',how='last')
wrongindex = yearEnd.index
#newindex = wrongindex.year+1
#newindex
#yearEnd.index = pd.to_datetime(['01/01/2005','01/01/2006')
#yearStart = pif_time.resample('AS-JAN',how='first')
                  #, how='diff')
#yearEnd.index = pd.to_datetime(newindex,format='%Y')
#yearEnd['01001'].plot(ms='o',ls='--',color='midnightblue')
yearEnd.diff()['01001'].plot()

### AJR 11.21.2014

# Data Prep for ...
##1. Agent AFO listing
##2. mapping zipcodes to FIPSCDs

In [ ]:
import os
#os.chdir('../../projects/Production Migration Project/csv')

In [ ]:
# load the zipCode database
zipDF = pd.read_csv('zip_code_database.csv',dtype={'zip':np.str})
zipDF.head()

In [ ]:
zipDF.decommissioned.sum()

In [ ]:
zipDF.county.unique()

In [ ]:
zipDF.estimated_population.hist(bins=30)

In [ ]:
sum(zipDF.county.isnull())

In [ ]:
print zipDF.state.unique(), len(zipDF.state.unique())
nonUSstateCodes = ['AA','AE','AP','AS','FM','GU','MH','MP','PR','PW','VI']

In [ ]:
for a in np.sort(zipDF[~zipDF['state'].isin(nonUSstateCodes)].state.unique()):
    print a, zipDF[zipDF.state == a].timezone.unique()

In [ ]:
np.sort(zipDF[~zipDF['state'].isin(nonUSstateCodes)].state.unique()), len(zipDF[~zipDF['state'].isin(nonUSstateCodes)].state.unique())

In [ ]:
# remove zip codes for 'states' outside the US: i.e.
zipDF = zipDF[~zipDF['state'].isin(nonUSstateCodes)]
zipDF.index = np.arange(0,len(zipDF))
zipDF.head()

In [ ]:
#list of missing zipCode Counties
print len(zipDF[zipDF['county'].isnull()])
zipDF[zipDF['county'].isnull()]
# map these to nearbly county


In [ ]:
## fix these mising counties in zipDF
zipCountyMissing = zipDF[zipDF['county'].isnull()]
print len(zipCountyMissing.primary_city.unique()), len(zipCountyMissing.state.unique())
#for line in zipDF[zipDF['county'].isnull()]:
#    print line
zipCountyMissing[['zip','primary_city','state','decommissioned']]

In [ ]:
zipCountyMissing.primary_city.unique(), zipCountyMissing.zip.unique()

In [ ]:
# define a dictionary for these 'bad' cases
# pulled from looking up on http://www.zip-codes.com/zip-code/22350/zip-code-22350.asp 
#I should map from zip to county too?
missingCountyFixDict = {}
missingCountyFixDict['New York']='New York County'
missingCountyFixDict['Pittsburgh']='Allegheny County'
missingCountyFixDict['Washington'] ='District of Columbia'
missingCountyFixDict['Alexandria'] ='Alexandria City'
missingCountyFixDict['Woodberry Forest'] ='Madison County'
missingCountyFixDict['High Shoals'] ='Oconee County'
missingCountyFixDict['Rhodhiss'] = 'Caldwell County'
missingCountyFixDict['Kennesaw'] ='Cobb County'
missingCountyFixDict['Miami']='Miami-Dade County'
missingCountyFixDict['Elbridge'] ='None'
missingCountyFixDict['Tougaloo'] ='Hinds County'
missingCountyFixDict['Des Moines']='Polk County'
missingCountyFixDict['Freeman Spur']='Williamson County'
missingCountyFixDict['Farmerville']='Union Parish'
missingCountyFixDict['Hinton']='Caddo County' #judgement call based upon wikipedia: spans both Caddo and Canadian Counties
missingCountyFixDict['Ardmore']='Carter County'
missingCountyFixDict['Frisco']='Collin County' #judgement call based upon wikipedia: majority seems to be here rather than Denton County
missingCountyFixDict['Stonington']='None'
missingCountyFixDict['Salt Lake City']='Salt Lake County'

missingCountyFixDict['Kings Canyon National Pk'] = 'Tulare County'
missingCountyFixDict['Springfield']='Lane County'
missingCountyFixDict['Mill Creek']='Snohomish County'
missingCountyFixDict['Seattle']= 'King County'
missingCountyFixDict['Wasilla'] = 'Matanuska-Susitna Borough'
#'New York', 'Pittsburgh', 'Washington', 'Alexandria',
#       'Woodberry Forest', 'Rhodhiss', 'High Shoals', 'Kennesaw', 'Miami',
#       'Elbridge', 'Tougaloo', 'Des Moines', 'Freeman Spur', 'Farmerville',
#       'Hinton', 'Ardmore', 'Frisco', 'Stonington', 'Salt Lake City',
#       'Kings Canyon National Pk', 'Springfield', 'Mill Creek', 'Seattle',



In [ ]:
for city,state in zipCountyMissing[['primary_city','state',]].values:
    print city, state
    countyResults = zipDF[(zipDF.primary_city == city) & (zipDF.state == state)].county.unique()
    print countyResults

In [ ]:
len(missingCountyFixDict)

In [ ]:
zipDF.ix[zipCountyMissing.index]['county'] =zipDF.ix[zipCountyMissing.index].primary_city.apply(lambda x: missingCountyFixDict[x])

In [ ]:
zipDF.loc[zipDF['county'].isnull(),'county'] = zipDF[zipDF['county'].isnull()].primary_city.apply(lambda x: missingCountyFixDict[x])

In [ ]:
zipDF.to_csv('us_zip_cod_listing1.csv')

In [ ]:
len(zipDF.county.unique())

In [ ]:
len(fipsCountyList.County.unique())

In [ ]:
fipsCountyList

In [ ]:
len(fipsCountyList)

In [ ]:
## load file to map county name to FIPS code
nflow1011 = pd.read_csv('countynetflow_1011.csv',dtype={'State_Code':np.str,
                                'County_Code':np.str,
                                'Code1':np.str,
                                'Code2':np.str,
                                'Abbrv':np.str,
                                'Name':np.str})

In [ ]:
nflow1011.columns

In [ ]:
#drop unnecessary columns
nflow1011.drop(['Unnamed: 0','Return_In', 'Return_Out', 'Return_Net', 'Exmpt_In', u'Exmpt_Out', u'Exmpt_Net', u'Aggr_AGI_In', u'Aggr_AGI_Out', u'Aggr_AGI_Net'],1,inplace=True)
# assign FIPS_CD
nflow1011['FIPS_CD'] = nflow1011.State_Code + nflow1011.County_Code
# assign ITEM_CD (i.e. migrants or non-migrants, etc.) --> 
# I just want the  non_migrants (ITEM_CD == FIPS_CD) 
nflow1011['ITEM_CD'] = nflow1011.Code1 + nflow1011.Code2

# keep only cases where 'ITEM_CD' == FIPS_CD
print "keep only {0} FIPSCDS".format(len(nflow1011[nflow1011['ITEM_CD']==nflow1011['FIPS_CD']]))
fipsList = nflow1011[nflow1011['ITEM_CD']==nflow1011['FIPS_CD']].copy()
fipsList.index = np.arange(0,len(fipsList))
fipsList.head()


In [ ]:
## create a dictionary that maps FIPS_CD to Name --> remove the Non-Migrants part
fipsList['County'] = fipsList.Name.apply(lambda x: x.split(' Non-Migrants')[0])


In [ ]:
# split on ' '
def countyNameNorming(x):
    nmissing = 0
    y = x.split(' ') #assign y the space split county name#
    # look at the last element of this list
    # standard names are "County, Borough, and Parish"
    stdNames = ['County','Borough','Parish','city']
    if y[-1] in stdNames:
        return x #don't change
    else:
        if y[-1].startswith('ci'):# =='City':# or y[-1] == 'city':
            y[-1] = 'city' # works for cities in virginia
            #print "\t", x
        elif y[-1].startswith('Co'):
            y[-1] = 'County'
        elif y[-1].startswith('Bo'):
            y[-1] = 'Borough'
        elif y[-1].startswith('Pa'):
            y[-1] = 'Parish'#
        elif y[-1].startswith('Are'):
            #print x, y[-1]
            y[-1] = 'Area'
        elif y[-1].startswith('Ce'):
            y[-1] = 'Census Area'
        elif y[-1].startswith('c'):
            y[-1] = 'city'
            #y[-1] = 'County' # assumes a county
            print x, y[-1]
            #print x, y[-1]
            
        
        else:
            nmissing+=1
            print "\t unknown term\t", x
            
            
        z=' '.join(y)
        return z
    
    

In [ ]:
fipsList['County'] = fipsList['County'].apply(lambda x: countyNameNorming(x))

In [ ]:
len(fipsList[fipsList['Abbrv']=='AK']['County'].unique()), len(zipDF[zipDF.state=='AK'].county.unique())


In [ ]:
fipsList[fipsList['Abbrv']=='AK']['County'].unique()

In [ ]:
akset1 = set(zipDF[zipDF.state=='AK'].county.unique())
akset2 = set(fipsList[fipsList['Abbrv']=='AK']['County'].unique())
akset1.difference(akset2)

In [ ]:
zipDF[zipDF.state=='AK' ][['zip','county','primary_city','decommissioned']]

In [ ]:
akset2.difference(akset1)

In [ ]:
# create an fips dictonary of changes
fipsCountyNamesDict = {}
fipsCountyNamesDict['Aleutians East'] = 'Aleutians East Borough'
fipsCountyNamesDict['Aleutians West'] = 'Aleutians West Census Area'
fipsCountyNamesDict['Anchorage Munic'] = '',
 'Bethel Census A',
 'Fairbanks North',
 'Hoonah Angoon C',
 'Juneau City and',
 'Kenai Peninsula',
 'Ketchikan Gatew',
 'Kodiak Island B',
 'Lake and Penins',
 'Matanuska Susit',
fipsCountyNamesDict['Northwest Arcti'] = 'Northwest Arctic Borough'
fipsCountyNamesDict['Petersburg Census Area'] = 'Petersburg Borough'
 'Prince of Wales',
 'Sitka City and',
 'Skagway Municip',
 'Southeast Fairb',
 'Valdez County',
 'Wrangell City a',
 'Wrangell Peters',
 'Yakutat City an',
 'Yukon Koyukuk C']
    
    
 
 'Anchorage Borough',
 'Bethel Census Area',
 'City and Borough of Juneau',
 'Fairbanks North Star Borough',
 'Hoonah-Angoon Borough',
 'Juneau Borough',
 'Kenai Peninsula Borough',
 'Ketchikan Gateway Borough',
 'Kodiak Island Borough',
 'Lake and Peninsula Borough',
 'Matanuska-Susitna Borough',
 'Municipality of Anchorage',
 ,
 
 'Prince of Wales-Outer Ketchikan Borough',
 'Sitka Borough',
 'Skagway Borough',
 'Southeast Fairbanks Census Area',
 'Valdez-Cordova Census Area',
 'Wrangell Borough',
 'Yakutat Borough',
 'Yukon-Koyukuk Census Area'

In [ ]:
fipsList[fipsList.Abbrv=='AK']

In [ ]:
[x for x in zipDF[zipDF.state =='AK'].county.unique() if str(x).startswith('Peters')]

In [ ]:
zipDF[zipDF.state =='AK'].county.unique()

In [ ]:
zipDF[zipDF.county=='Prince of Wales-Outer Ketchikan Borough']

In [ ]:
%ls

##
# load the fips-code listing from wikipedia

In [ ]:
fipsCountyList = pd.read_csv('fipsListing_1.csv',dtype={'INCITS':np.str})
# fix the INCITS CODES to prepend 0 if smaller
fipsCountyList['INCITS']=fipsCountyList['INCITS'].apply(lambda x: prependZeros(x,5))
fipsCountyList.head()

In [ ]:
fipsCountyList[fipsCountyList.State == 'Alaska']

In [ ]:
badcases = [c for c in fipsCountyList.County if c.endswith(']')]
for b in badcases:
    match = re.match('(.+)(\[\d+\])',b)
    if match:
        print match.group(1)

In [ ]:
len(fipsCountyList.State.unique())

In [ ]:
##
state2abbrv = {}
listOfstateNames = fipsCountyList.State.unique()
abbrv = [ 'AL','AK','AZ','AR',
 'CA', 'CO', 'CT', 'DE', 'DC',
 'FL', 'GA', 'HI', 'ID','IL','IN','IA',
 'KS', 'KY', 'LA',
 'ME','MD','MA','MI', 'MN',  'MS','MO', 'MT',
 'NE', 'NV',  'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 
 'OH', 'OK', 'OR', 'PA',
 'RI', 'SC', 'SD',
 'TN', 'TX', 'UT', 'VT', 'VA',
 'WA', 'WV', 'WI', 'WY']
for i in xrange(0,len(listOfstateNames)):
    state2abbrv[listOfstateNames[i]]=abbrv[i]


In [ ]:
# append stAbbrev to fipsCountyList
fipsCountyList['stAbbv'] = [state2abbrv[x] for x in fipsCountyList.State.values]

In [ ]:
import re
def fix_countyAnnotations(x):
#for c in fipsCountyList.County:
    match = re.match('(.*)(\[\d*\])',x)
    if match:
        return match.group(1)#[:-1]
    else:
        return x
        #print c, match.group(1)[:-1]
        
fipsCountyList['County'] = fipsCountyList.County.apply(lambda x: fix_countyAnnotations(x))

In [ ]:
zipDF[zipDF.primary_city=='San Francisco'].county.unique()

In [ ]:
hasCommas = [c for c in fipsCountyList.County if ',' in c]
len(hasCommas)

In [ ]:
hasCommas

In [ ]:
def fixCandBof(x):
    y = x.split(',')
    if len(y)>1:
        print y

In [ ]:
fipsCountyList.loc[67:95,'County'].apply(lambda x: fixCandBof(x))

In [ ]:
zipDF[zipDF.state == 'AK']['county'].unique()


In [ ]:
fipsCountyList.County.str.replace('?','-',inplace=True)

In [ ]:
[a for a in fipsCountyList.loc[67:90,'County'] if '?' in a]

In [ ]:
fipsCountyList.loc[67:95,'County']#.str.replace('?', '-')

In [ ]:
zipDF[zipDF.primary_city=='Anchorage']

In [ ]:
[c for c in fipsCountyList.County if ',' in c]

In [ ]:
endswithCity = [c for c in fipsCountyList.County if c.endswith(', City of')]
for city in endswithCity:
    rename = city.split(',')
    newName = rename[0]+' City'
    rowID = fipsCountyList[fipsCountyList.County==city].index 
    fipsCountyList.loc[rowID,'County']=newName

[c for c in fipsCountyList.County if c.endswith(', City of')]    

In [ ]:
fipsCountyList['stcnty']=fipsCountyList['stAbbv']+fipsCountyList['County']
fips_STcounty = fipsCountyList.stcnty.unique()

In [ ]:
zipDF.columns

In [ ]:
zipDF['stcnty']=zipDF['state']+zipDF['county']
zip_code_STcounty = zipDF['stcnty'].unique()

In [ ]:
''.join(hasDot[2].split('.'))

In [ ]:
zipDF[zipDF['county'].isnull()]

In [ ]:
hasDot2 = [c for c in zipDF.county if '.' in c]
for c in hasDot2:
    rename = ''.join(c.split('.'))
    rowID = zipDF[zipDF.county==c].index 
    zipDF.loc[rowID,'county']=rename

In [ ]:
hasDot = [c for c in fipsCountyList.County if '.' in c]
for c in hasDot:
    rename = ''.join(c.split('.'))
    rowID = fipsCountyList[fipsCountyList.County==c].index 
    fipsCountyList.loc[rowID,'County']=rename

In [ ]:

print len(zip_code_STcounty), len(fips_STcounty)
#import re
#map(lambda x: if re.match('(.*)\s team:', x, re.IGNORECASE).group(1)[:-1],fipsCountyList.County else x)

In [ ]:
set(zip_code_STcounty).difference(set(fips_STcounty))

In [ ]:
fipsCountyList[fipsCountyList.stAbbv=='WI']

In [ ]:
print len([x for x in fipsCountyList.County.unique() if x.endswith('Parish')])
print len()

In [ ]:
len(fipsList[fipsList['Abbrv']=='AK']['County'])

In [ ]:
nmissing = 0
fipsList['County'].apply(lambda x: countyNameNorming(x))

In [ ]:
print len([x for x in fipsList['County'].values if x.split(' ')[-1].startswith('Pa')])
print len(fipsList[fipsList['Abbrv']=='LA'])

In [ ]:
[x for x in fipsList[fipsList['Abbrv']=='LA'].County if not (x.split(' ')[-1].endswith('P'))]


## load data from pcadss and from ADM

In [ ]:
agentDF = pd.read_csv('agent/agentAFOfull.tsv',delimiter='\t') # this is the data from PCADSS
agentDF.columns

In [ ]:
agentDF.AGENT_STATUS.value_counts()

In [ ]:
# load and mask data from ADM
agentList = pd.read_csv('agent/agentInfo.csv', dtype={'CUR PLAN CODE':np.str, 'MOA ST-AGT-CD':np.str,'PRIMARY ST-AGT-CD.1':np.str})

In [ ]:
len(agentList.SSN.unique()), len(agentList['AWARDS ID'].unique())
#agentList.head()

In [ ]:
#from collections import defaultdict
#create a hash keytable for the SSN and remove them from the dataset
alhash= {}
hstart = 20
for v in agentList.SSN.unique():
    alhash[v]=hstart+1
    hstart+=1
    
# assign the agentList SSN to unique ids
agentList['uniqID'] = agentList['SSN'].apply(lambda x: alhash[x])
# drop the SSN values
agentList.drop('SSN',axis=1,inplace=True)
# create a short 5-digit zip code value
agentList['zip5'] = agentList['OFFICE ZIP'].apply(lambda x: x[:5])

print np.shape(agentList)



In [ ]:
# remove Canadian cases:
nCanadian = len(agentList[agentList['GEO MARKET AREA']=='Canada'])
canadianState1 = agentList[agentList['GEO MARKET AREA']=='Canada']['STATE.1'].unique()
print nCanadian," Canadian agents for these provinces will be dropped:", canadianState1
agentList = agentList[~agentList['STATE.1'].isin(canadianState1)]
print np.shape(agentList)
#agentList.replace('STATE.1','XX')#drop('STATE.1')

In [ ]:
agentList['GEO MARKET AREA'].value_counts()

## things to do:
1. assign the fipsCode to each agent row
2. get the dates when an agent is active so I can do a simple groupby-count to get number of active 

In [ ]:
agentList[[u'APPOINT DATE', u'FIRST APPOINT DATE', u'STATUS','TERMINATION DATE','zip5']].head()

In [ ]:
# how many unique zip5 codes?
len(agentList['zip5'].unique())

In [ ]:
agentList['STATE.1'].value_counts()

In [ ]:
agentList.to_csv('../../../agentInfo2.csv')

In [ ]:
# list which agents have mulitple lines
aLidsVC = agentList.uniqID.value_counts()
aLidsVC[aLidsVC > 1]

In [ ]:
del alhash

In [ ]:
agentList[agentList.uniqID==15143][['NAME','CODE',u'APPOINT DATE', u'FIRST APPOINT DATE', u'STATUS','TERMINATION DATE','zip5','STATE.1','REASON TERMINATED']]#['STATE.1','OFFICE ZIP']]

In [ ]:
# check if there are enough 'terminations for these duplicate list of ids
dupIds = [c for c in aLidsVC.index]
dupIds[:10]

In [ ]:
agentList['APPOINT DATE'].min(), agentList['FIRST APPOINT DATE'].min()

In [ ]:
dec2014 = pd.to_datetime('01DEC2014')
firstDate = pd.to_datetime(agentList['APPOINT DATE'].min())
firstDate, dec2014


In [ ]:
dateSpan = pd.date_range(start ='4/1/1929',end='12/1/2014',freq='M')
len(dateSpan)

In [ ]:
len(agentList.zip5.unique())

In [ ]:
zipDF.columns

In [ ]:
[c for c in zipDF.county if c.endswith('Parish')]

In [ ]:
# create a zip to county Mapping
zipToCounty = {}
for z in zipDF.zip:#.ix[:10]:
    #print z, zipDF[zipDF.zip == z].county
    zipDF[z] = zipDF[zipDF.zip == z].county

In [ ]:
def activeCount(x,t):
    

In [ ]:
for id in dupIds[:10]:
    print id
    print "----------------------"
    print agentList[agentList.uniqID==id][['NAME','CODE',u'APPOINT DATE', u'FIRST APPOINT DATE', u'STATUS','TERMINATION DATE','zip5','STATE.1','REASON TERMINATED']]#['STATE.1','OFFICE ZIP']]

In [ ]:
alIDtermCD = agentList.groupby('uniqID')['REASON TERMINATED'].unique()


In [ ]:
vclisting_byID = alIDtermCD.value_counts()

In [ ]:
print len(vclisting_byID)

In [ ]:
zcheck = agentList[(agentList['STATE.1']=='AK') & (agentList['TERMINATION DATE'].isnull())]['zip5'].values
zcheck2 = agentList[agentList['STATE.1']=='AK']['zip5'].unique()
len(zcheck2), len(zcheck)

In [ ]:
for z in zcheck2:
    print zipDF[zipDF.zip==z]['county'].values

In [ ]:
zlist = agentList['zip5'].unique()
print len(zlist)

In [ ]:
print len(zipDF)

In [ ]:
zzlist = zipDF[zipDF.zip.isin(zlist)].zip.unique()
print len(zzlist)

In [ ]:
extrazips_fromADM = list(set(zlist).difference(set(zzlist)))

In [ ]:
agentList[agentList.zip5.isin(extrazips_fromADM)][['STATE.1','NAME','TERMINATION DATE','zip5']]

In [ ]:
agentList[(agentList.zip5.isin(extrazips_fromADM)) & (pd.to_datetime(agentList['TERMINATION DATE']) >pd.to_datetime('01JAN2003'))][['STATE.1','NAME','TERMINATION DATE','zip5']]

In [ ]:
agentList.ix[25284]

In [ ]:
agentList=

In [ ]:
dateutil.parser.parse(agentList.loc[35139,'TERMINATION DATE'])

In [ ]:
pd.to_datetime(agentList.loc[35139,'TERMINATION DATE'],format='%d%M%Y')

In [ ]:
zipDF[zipDF.zip=='97003']

In [ ]:
# reread from new file
aa = pd.read_csv('../../../agentInfo2.csv', dtype={'CUR PLAN CODE':np.str, 'MOA ST-AGT-CD':np.str,'PRIMARY ST-AGT-CD.1':np.str,'zip5':np.str})#, parse_dates = dateCols)
#for c in aa.columns:
#    if 'DATE' in c:
#        print c

In [ ]:
dateCols = []
for c in aa.columns:
    if 'DATE' in c:
        dateCols.append(c)
        
dateCols.remove('EARLY NOTIFICATION PROGRAM DATE')

In [ ]:
agentList['APPOINT DATE'].values

In [ ]:
pd.to_datetime('01SEP1958',format='%d%b%Y')

In [ ]:
aa[dateCols] = aa[dateCols].applymap(lambda x: pd.to_datetime(x,format='%d%b%Y'))

In [ ]:
jan2002start = pd.to_datetime('31DEC2001',format='%d%b%Y')
len(aa[aa['TERMINATION DATE'] < jan2002start])

In [ ]:
agentList[(agentList.zip5.isin(extrazips_fromADM)) & (pd.to_datetime(agentList['TERMINATION DATE'],format='%d%b%Y') >pd.to_datetime('01JAN2002',format='%d%b%Y'))][['STATE.1','NAME','TERMINATION DATE','zip5']]

In [ ]:
aa[aa['TERMINATION DATE'].isnull()][['NAME','TERMINATION DATE']]

In [ ]:
z91910 = aa[aa.zip5=='91910']
#aa[(aa.zip5 == 91910) & (aa['TERMINATION DATE'] > pd.to_datetime('12/31/2009')) & (aa['APPOINT DATE'] < pd.to_datetime('01/01/2010'))][['STATE.1','NAME','TERMINATION DATE','zip5']]
len(z91910)

In [ ]:
def isActive(df,date,today=pd.to_datetime('12-31-2020')):
    #print sum(df['TERMINATION DATE'].isnull())
    # replace null times with today
    df['TERMINATION DATE'].fillna(today,inplace=True)
    #print sum(df['TERMINATION DATE'].isnull())
    nactive= len(df[(df['APPOINT DATE'] <= date) & (df['TERMINATION DATE']>date)])
    #print nactive
    return nactive
    #print len(df1)
    #print df1[['NAME','APPOINT DATE','TERMINATION DATE']]

In [ ]:
isActive(z91910,pd.to_datetime('08-01-1995'))

In [ ]:
z91910[['NAME','APPOINT DATE','TERMINATION DATE']]

In [ ]:
atemp = aa[aa['STATE.1']=='ME']
len(atemp)

In [ ]:
agentList[agentList['STATE.1']=='ME'].zip5.values

In [ ]:
agtZipDF = pd.DataFrame(columns=['zipCode'])
#State_Code','County_Code',
#      'Code1','Code2',
#      'Abbrv','Name',
#      'Return_In','Return_Out','Return_Net',
#      'Exmpt_In','Exmpt_Out','Exmpt_Net',
#      'Aggr_AGI_In','Aggr_AGI_Out','Aggr_AGI_Net'])
#    return df
for z,df in atemp.groupby('zip5'):
    #agtZipDF = pd.DataFrame(columns=['zipCode'])
    print z, len(df), isActive(df,pd.to_datetime('08-01-2001'))
    print 
    
    
    

In [ ]:
df[['TERMINATION DATE','APPOINT DATE']]

In [ ]:
dateSpan[-20:]

In [ ]:
dateSpan[-12], z

In [ ]:
agtActive = pd.DataFrame(columns=[z])
agtActive

In [ ]:
z = '91910'
for t in dateSpan:
    agtActive.loc[t,z] = isActive(z91910,t)

In [ ]:
agtActive['91910'].plot()

In [ ]:
np.shape(agtActive)

In [ ]:
agtActive.columns

In [ ]:
agtActive.loc[dateSpan[-30:],:]

In [ ]:
yearSpan1 = pd.date_range(start ='1/1/1999',end='1/1/2015',freq='A')
len(yearSpan1)

In [ ]:
yearSpan1

In [ ]:
for t in yearSpan1:
    print t, isActive(z91910,t)

In [ ]:
aa.zip5.head()

In [ ]:
YrlyAgtActive = pd.DataFrame(columns=['03431'])
for z,df in aa.groupby('zip5'):
    print z
    for t in yearSpan1:
        YrlyAgtActive.loc[t,z]=isActive(df,t)

In [ ]:
print np.shape(YrlyAgtActive)

In [ ]:
YrlyAgtActive[['03431','03110']].tail()

In [ ]:
YrlyAgtActive.to_csv('yrlyActiveAgents_byZip.csv')

In [ ]:
dateSpan1 =  pd.date_range(start ='1/1/1999',end='1/1/2015',freq='M')
len(dateSpan1)


In [ ]:
MtlyAgtActive=pd.DataFrame(columns=['03110'])
for z, df in aa.groupby('zip5'):
    print z
    for t in dateSpan1:
        MtlyAgtActive.loc[t,z]=isActive(df,t)

In [ ]:


def get_data_for_fips(df, fips_state, fips_county):
    return df[np.array(df[[0]] == fips_state) & np.array(df[[1]]==fips_county)]

def get_totals_for_fips(df, fips_state, fips_county):
    ff = df[np.array(df[[0]] == fips_state) & np.array(df[[1]]==fips_county)]
    return ff.select(lambda x: int(ff.loc[x,ff.columns[2]]) >= 96 or (ff.loc[x,ff.columns[2]] == ff.loc[x,ff.columns[0]] and ff.loc[x,ff.columns[3]] == ff.loc[x,ff.columns[1]])).sort([ff.columns[2],ff.columns[3]])

def net_df():
    df = pd.DataFrame(columns=['State_Code','County_Code',
      'Code1','Code2',
      'Abbrv','Name',
      'Return_In','Return_Out','Return_Net',
      'Exmpt_In','Exmpt_Out','Exmpt_Net',
      'Aggr_AGI_In','Aggr_AGI_Out','Aggr_AGI_Net'])
    return df

def get_net_change_for_fips(in_df, out_df, fips_state, fips_county):
    in_df = get_totals_for_fips(in_df, fips_state, fips_county).reset_index()
    out_df = get_totals_for_fips(out_df, fips_state, fips_county).reset_index()
    assert(np.all(np.array(in_df[[3,4]])==np.array(out_df[[3,4]])))
    
    df = net_df()
    df[['State_Code','County_Code',
      'Code1','Code2',
      'Abbrv','Name']] = in_df[['State_Code_Dest','County_Code_Dest',
                                              'State_Code_Origin','County_Code_Origin',
                                              'State_Abbrv','County_Name']]
    #df['County_Code'] = in_df['County_Code_Dest']
    #State_Code_Origin	County_Code_Origin	State_Abbrv	County_Name
    
    df['Return_In'] = in_df['Return_Num']
    df['Return_Out'] = out_df['Return_Num']
    df['Return_Net'] = in_df['Return_Num'] - out_df['Return_Num']

    df['Exmpt_In'] = in_df['Exmpt_Num']
    df['Exmpt_Out'] = out_df['Exmpt_Num']
    df['Exmpt_Net'] = in_df['Exmpt_Num'] - out_df['Exmpt_Num']
    
    
    df['Aggr_AGI_In'] = in_df['Aggr_AGI']
    df['Aggr_AGI_Out'] = out_df['Aggr_AGI']
    df['Aggr_AGI_Net'] = in_df['Aggr_AGI'] - out_df['Aggr_AGI']
    
    return df

    
def get_all_fips(df):
    fips = df[[0]].fillna(value='01').values + df[[1]].fillna(value='001').values
    s = set(fips[:,0].tolist())
    return set(map(lambda x: (x[0:2], x[2:6]), s))

def load_flow_csv(file_name):
    return  pd.read_csv(file_name,
                         dtype={'State_Code_Dest':np.str,
                                'County_Code_Dest':np.str,
                                'State_Code_Origin':np.str,
                                'County_Code_Origin':np.str,
                                'State_Abbrv':np.str,
                                'County_Name':np.str})

In [ ]:
agtProd = pd.read_csv('../../Notes/agentProd.csv', dtype={'CUR PLAN CODE':np.str, 'MOA ST-AGT-CD':np.str,'PRIMARY ST-AGT-CD.1':np.str,'zip5':np.str})

In [ ]:
print agtProd.columns[2:13]
agtProd[agtProd.columns[2]].dtype

In [ ]:
agtProd[agtProd.columns[2:13]].applymap(lambda x: float(x))

In [ ]:
agtProd = pd.read_csv('../../Notes/agentProd.csv', thousands=',' ,dtype={'SSN':np.str})

In [ ]:
agentList['AWARDS ID'].value_counts()

In [ ]:
agentList.columns

In [ ]:
agentList[agentList['AWARDS ID'] == 110778]

In [ ]:
fireCols

In [ ]:
y2008 = [c for c in agtProd if c.endswith('2008')]

In [ ]:
y2008

In [ ]:
fireY1

In [ ]:
agtProd[agtProd.AWARDS_ID == 110778][y1]

In [ ]:
agtProd[agtProd.AWARDS_ID.isnull()][agtProd.columns[2:]].sum()
agtProd[agtProd.AWARDS_ID.isnull()]

In [ ]:
## join AGENTList with agentProd


In [ ]:
# read in the agent file
agentList = pd.read_csv('agent/agentInfo2.csv', dtype={'CUR PLAN CODE':np.str, 'MOA ST-AGT-CD':np.str,'PRIMARY ST-AGT-CD.1':np.str,'zip5':np.str},index_col=0)

In [ ]:
len(agentList['AWARDS ID'].unique())

In [ ]:
len(agentList)

In [ ]:
agtProd.drop('SSN',axis=1,inplace=True) # remove SSN

In [ ]:
agentList.rename(columns={'AWARDS ID':'AWARDS_ID'},inplace=True) #get joining axis to match


In [ ]:
agentList.index.is_unique

In [ ]:
agentList.AWARDS_ID.value_counts()

In [ ]:
len(agtProd.AWARDS_ID.unique())

In [ ]:
pd.merge?

In [ ]:
df = pd.merge(agentList,agtProd,on='AWARDS_ID',how='left')

In [ ]:
len(df)

In [ ]:
len(df.AWARDS_ID.unique())

In [ ]:
volAutoCols = [c for c in df.columns if c.startswith('VOL AUTO')]
print volAutoCols
volAutoCols.remove('VOL AUTO 2013-11 PYTD')
volAutoCols.remove('VOL AUTO 2014-11 YTD')
volAutoCols

In [ ]:
def getAnnualColumnNames(df, cname):
    columnList = [c for c in df.columns if c.startswith(cname)]
    # id elements that are PYTD/YTD
    toDropList = [c for c in columnList if c.endswith('YTD')]
    for a in toDropList:
        columnList.remove(a)
    return columnList

In [ ]:
rawnewAutoCols= getAnnualColumnNames(df,'AUTO RAW')
rawnewAutoCols

In [ ]:
# consider the cases where someone changes locations
df.AWARDS_ID.value_counts().head()

In [ ]:
df.columns[:30]

In [ ]:
df[df.AWARDS_ID==119365][['APPOINT DATE','TERMINATION DATE','STATUS','zip5','PRIMARY ST-AGT-CD','VOL AUTO 2005','VOL AUTO 2008']]

In [ ]:
# convert TERMINATION DATE to datetime
isActiveIDX = df[df['TERMINATION DATE'].isnull()].index

print len(isActiveIDX), " active agents"


In [ ]:
df['TERMINATION DATE'] = df['TERMINATION DATE'].apply(lambda x: pd.to_datetime(x,format='%d%b%Y'))

In [ ]:
ended2005orLaterIDX = df[df['TERMINATION DATE'] > '2004-12-31'].index
len(ended2005orLaterIDX)

In [ ]:
keepIDX = isActiveIDX.union(ended2005orLaterIDX)
len(keepIDX)

In [ ]:
df[df['TERMINATION DATE'].apply(lambda x: pd.to_datetime(x,format='%d%b%Y')) > '2004-12-31']['TERMINATION DATE']

In [ ]:
# do the same for RAW New
rawnewAutoCols = [c for c in df.columns if c.startswith('AUTO RAW')]
print rawnewAutoCols

In [ ]:
df.columns[200:]

# I should go through this one and check the year with the termination date for cases with multiple listings -- also need to look at apoint date

In [ ]:
kdf = df.ix[keepIDX].copy()
print len(kdf)
kdf.index = np.arange(0,len(kdf)) # reindex it

In [ ]:
# convert appoint date to datetime
kdf['APPOINT DATE'] = kdf['APPOINT DATE'].apply(lambda x: pd.to_datetime(x,format='%d%b%Y'))

In [ ]:
nvc = kdf.AWARDS_ID.value_counts() # which ones are multiple valuecounts
print len(nvc[nvc>1])
dupAgentIDs = nvc[nvc>1]

In [ ]:
rawnewAutoCols

In [ ]:
yearSpan1 = pd.date_range(start ='1/1/2004',end='1/1/2014',freq='A')
print len(yearSpan1),len(rawnewAutoCols)

In [ ]:
[t for t in yearSpan1]

In [ ]:
kdf[kdf.AWARDS_ID == 119365]['TERMINATION DATE']

In [ ]:
kdf[kdf.AWARDS_ID == 122000].index

In [ ]:
[t for t in yearSpan1]

In [ ]:
kdf.ix[8049]['TERMINATION DATE'] 

In [ ]:
dupAgentIDs = map(int,dupAgentIDs.index)


In [ ]:
dupAgentIDs[5]

In [ ]:
kdf[kdf.AWARDS_ID == dupAgentIDs[5]][['APPOINT DATE','TERMINATION DATE']]

In [ ]:
kdf[kdf.AWARDS_ID == dupAgentIDs[5]]

In [ ]:
yearCols = {}
years2 = ['2005','2006','2007','2008','2009','2010','2011','2012','2013']
for y in years2:
    yearCols[y]= [b for b in kdf.columns if y in b]

In [ ]:
# clean up the 2013 PYTD values
yy = yearCols['2013']
for a in yy:
    print a
    if a.endswith('PYTD'):
        print a
        yy.remove(a)
    
len(yearCols['2013']), len(yy)

## approach to correct for duplicated/moved agents
* assigned the columns for a given year to the corresponding dictionary `yearCol['2011']` for example
* copy singletons to a new data frame unchanged.
* step through the duplicated agents

In [ ]:
fullAGT = kdf[~kdf.AWARDS_ID.isin(dupAgentIDs)].copy()
len(fullAGT), len(kdf)

In [ ]:
minStart = yearSpan1[1]


In [ ]:
for a in dupAgentIDs:
    print a, 
    dindex = kdf[kdf.AWARDS_ID == a].index
    for idx in dindex:
        tdate = kdf.ix[idx]['TERMINATION DATE']
        adate = kdf.ix[idx]['APPOINT DATE']
        print idx, tdate, adate
        # zero out if appoint date Year is > year
        yrs_to_drop = [key for key in yearCols.keys() if int(key) < adate.year]
        
        if len(yrs_to_drop)>0:
            print yrs_to_drop
            for yy in yrs_to_drop:
                #pass
                kdf.loc[idx,yearCols[yy]]=0

        # zero out if year is > TERMINATION DATE
        tyear = tdate.year
        if tyear > -1:
            tyrs_to_drop = [key for key in yearCols.keys() if int(key) > tyear ]
            if len(tyrs_to_drop)>0:
                print "Tyrs to drop:", tyrs_to_drop
                for yy in tyrs_to_drop:
                    #pass
                    kdf.loc[idx,yearCols[yy]]=0

    

In [ ]:
kdf[kdf.AWARDS_ID==dupAgentIDs[8]][['APPOINT DATE','TERMINATION DATE','zip5','AUTO RAW NEW 2005','AUTO RAW NEW 2011','COUNTY','STATE']]

In [ ]:
kdf.to_csv('agent/agent_policy_amounts_1.csv')

In [ ]:
np.shape(kdf)

In [ ]:
bb = kdf.ix[idx][yearCols[yrs_to_drop[0]]].copy()
bb

In [ ]:
kdf.loc[idx,yearCols[yrs_to_drop[0]]]

In [ ]:
[key for key in yearCols.keys() if int(key)> 2006]

In [ ]:
def correct_for_agent_change(df,agent_id,year):
    # get the list of columns containing a given year
    yearCols = [b for b in df.columns if year in b]
    ydate = pd.to_datetime(year+'-12-31') # assign as the end of the year
    # identify cases where the termination date is before ydate
    # A) if year is greater than termination date reset values to zero
    df[df.AW]
    # B) if appoint date is greater than year reset values to zero
    #df[yearCols] = 
    tdate = df[df.AWARDS_ID == agent_id]['TERMINATION DATE']
    adate = df[df.AWARDS_ID == agent_id]['APPOINT DATE']
    # flag the ones with beginning PIF
    

In [ ]:
tdate = kdf[kdf.AWARDS_ID == dupAgentIDs[2]]['TERMINATION DATE']

In [ ]:
myY = '2009'
ydate = pd.to_datetime(myY+'-12-31')
ydate

In [ ]:
ydate

In [ ]:
kdf[(kdf.AWARDS_ID == dupAgentIDs[1]) & (kdf['TERMINATION DATE']< ydate)][['APPOINT DATE','TERMINATION DATE','zip5','COUNTY','STATE','STATUS','AUTO RAW NEW 2011']]#y0cols]

In [ ]:

for a in nvc.index[:3]:
    print a, 
    print kdf[kdf.AWARDS_ID == int(a)][['APPOINT DATE','TERMINATION DATE','zip5']]

In [ ]:
kdf[kdf.AWARDS_ID==119160][['APPOINT DATE', 'TERMINATION DATE']]

In [ ]:
kdf.groupby('zip5').get_group('46219')[lifeCols].sum()

In [ ]:
fireCols = getAnnualColumnNames(df,'FIRE')
fireCols, len(fireCols)

In [ ]:
lifeCols = getAnnualColumnNames(df,'LIFE')
lifeCols, len(lifeCols)

In [ ]:
healthCols = getAnnualColumnNames(df,'HEALTH')
#healthCols, len(healthCols)
phoenixCols = getAnnualColumnNames(df,'PHOENIX')
phoenixCols = phoenixCols[1:] #omit the agrement date

In [ ]:
len(kdf.columns)

In [ ]:
kdf.columns[1000:]

In [ ]:
def isActive(df,date,today=pd.to_datetime('12-31-2020')):
    #print sum(df['TERMINATION DATE'].isnull())
    # replace null times with today
    df['TERMINATION DATE'].fillna(today,inplace=True)
    #print sum(df['TERMINATION DATE'].isnull())
    nactive= len(df[(df['APPOINT DATE'] <= date) & (df['TERMINATION DATE']>date)])
    #print nactive
    return nactive